### Type hints powering schema validation

The schema that Pydantic validates against is generally defined by Python type hints.

Using type hints means that Pydantic integrates well with static typing tools like mypy and pyright and IDEs like pycharm and vscode

In [1]:
from typing import Annotated, Dict, List, Literal, Tuple

from annotated_types import Gt

from pydantic import BaseModel


class Fruit(BaseModel):
    name: str  
    color: Literal['red', 'green']  
    weight: Annotated[float, Gt(0)]  
    bazam: Dict[str, List[Tuple[int, bool, float]]]  


Fruit(name='Apple', color='red', weight=4.2, bazam={'foobar': [(1, True, 0.1)]})

Fruit(name='Apple', color='red', weight=4.2, bazam={'foobar': [(1, True, 0.1)]})

In [2]:
type(Fruit)

pydantic._internal._model_construction.ModelMetaclass

#### Supported types

Where possible Pydantic uses standard library types to define fields. 

For many useful applications, however, no standard library type exists, so Pydantic implements many commonly used types.

There are also more complex types that can be found in the "Pydantic Extra Types" package.

If no existing type suits your purpose you can also implement your own Pydantic-compatible types with custom properties and validation.

##### Standard Library Types

###### Booleans

 A standard bool field will raise a ValidationError if the value is not one of the following:

- A valid `boolean` (i.e. True or False),
- The integers 0 or 1,
- a `str` which when converted to lower case is one of '0', 'off', 'f', 'false', 'n', 'no', '1', 'on', 't', 'true', 'y', 'yes',
- a `bytes` which is valid per the previous rule when decoded to `str`

In [3]:
from pydantic import BaseModel, ValidationError


class BooleanModel(BaseModel):
    bool_value: bool


print(BooleanModel(bool_value=False))

print(BooleanModel(bool_value='False'))

print(BooleanModel(bool_value=1))

try:
    BooleanModel(bool_value='of')
except ValidationError as e:
    print(str(e))

bool_value=False
bool_value=False
bool_value=True
1 validation error for BooleanModel
bool_value
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value='of', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/bool_parsing


###### Datetime Types
- datetime.datetime
- datetime.date
- datetime.time
- datetime.timedelta

In [4]:
from datetime import datetime

from pydantic import BaseModel


class Event(BaseModel):
    dt: datetime = None


event = Event(dt='2032-04-23T10:20:30.400+02:30')
print(event)

dt=datetime.datetime(2032, 4, 23, 10, 20, 30, 400000, tzinfo=TzInfo(+02:30))


###### Number types
- int
- float
- enum.IntEnum
- decimal.Decimal
- Enum


###### Lists and Tuples
- list
- typing.List
- tuple
- typing.Tuple
- typing.NamedTuple

In [5]:
from typing import NamedTuple

from pydantic import BaseModel, ValidationError


class Point(NamedTuple):
    x: int
    y: int


class Model(BaseModel):
    p: Point


try:
    Model(p=('1.3', '2'))
except ValidationError as e:
    print(e)

1 validation error for Model
p.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='1.3', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/int_parsing


###### For more information of standard Library Types (https://docs.pydantic.dev/latest/api/standard_library_types/)

##### Pydantic types

###### Strict Types
Pydantic provides the following strict types, among others:

- StrictBool
- StrictBytes
- StrictFloat
- StrictInt
- StrictStr
- 
These types will only pass validation when the validated value is of the respective type or is a subtype of that type.

**StrictFloat = Annotated[float, Strict(True)]**

In [6]:
from pydantic import BaseModel, StrictFloat, ValidationError

class StrictFloatModel(BaseModel):
    strict_float: StrictFloat

try:
    StrictFloatModel(strict_float='1.0')
except ValidationError as e:
    print(e)

1 validation error for StrictFloatModel
strict_float
  Input should be a valid number [type=float_type, input_value='1.0', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/float_type


**PositiveInt = Annotated[int, Gt(0)]**

In [7]:
from pydantic import BaseModel, PositiveInt, ValidationError

class Model(BaseModel):
    positive_int: PositiveInt

m = Model(positive_int=1)
print(repr(m))
#> Model(positive_int=1)

try:
    Model(positive_int=-1)
except ValidationError as e:
    print(e.errors())

Model(positive_int=1)
[{'type': 'greater_than', 'loc': ('positive_int',), 'msg': 'Input should be greater than 0', 'input': -1, 'ctx': {'gt': 0}, 'url': 'https://errors.pydantic.dev/2.6/v/greater_than'}]


###### For more information of Pyndatic types (https://docs.pydantic.dev/latest/api/types/)

##### Custom types

Annotated is as a way to attach runtime metadata to types without changing how type checkers interpret them. Pydantic takes advantage of this to allow you to create types that are identical to the original type as far as type checkers are concerned, but add validation, serialize differently, etc.

In [8]:
from typing_extensions import Annotated

from pydantic import Field, TypeAdapter, ValidationError

CustomInt = Annotated[int, Field(gt=5)]

ta = TypeAdapter(CustomInt)

try:
    ta.validate_python(4)
except ValidationError as exc:
    print(exc)

1 validation error for constrained-int
  Input should be greater than 5 [type=greater_than, input_value=4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/greater_than
